In [2]:
import keras
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D,Flatten,Lambda
from keras.models import Model
import keras.backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
digit_input = Input(shape=(105,105,1))
x = Conv2D(64, (10, 10),activation='relu',input_shape=(105,105,1))(digit_input)
x = MaxPooling2D()(x)
x = Conv2D(128, (8, 8),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (6, 6),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (4, 4),activation='relu')(x)
out = Flatten()(x)
#out = Dense(4096,activation='sigmoid')(x)
vision_model = Model(digit_input,out)

letter1 = Input(shape=(105,105,1))
letter2 = Input(shape=(105,105,1))

out_1 = vision_model(letter1)
out_2 = vision_model(letter2)

def m_dist(A,B):
    return K.sum(K.abs(A-B),axis=1,keepdims=True)

merged_vector = Lambda(lambda x:m_dist(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])

out_fin = Dense(4096,activation="sigmoid")(merged_vector)
final_output = Dense(1,activation="sigmoid")(out_fin)

final_model = keras.models.Model(inputs=[letter1, letter2], outputs=final_output)

In [4]:
optimizer = SGD(lr=1e-3,momentum=0.5,decay=0,nesterov=False)

In [5]:
final_model.compile(optimizer=optimizer,
                   loss='binary_crossentropy',
                   metrics=['binary_accuracy'],)

In [6]:
final_model.layers[2].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 105, 105, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 128)       524416    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       589952    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 128)         0         
__________

In [7]:
from matplotlib import pyplot as plt 
%matplotlib inline
import glob
import cv2
import numpy as np
from collections import Counter
from IPython.core.display import display, HTML
import random

In [8]:
dataset_path="python\\images_background\\"
languages = [path.split("\\")[-1] for path in glob.glob(dataset_path+"*")]
all_alphabets = {language:[cv2.cvtColor(cv2.imread(image),cv2.COLOR_RGB2GRAY) for image in glob.glob(dataset_path+language+"/*/*")]\
                                                                  for language in languages }
lengthes = np.cumsum([len(all_alphabets[key]) for key in all_alphabets])  
lengthes = np.insert(lengthes,0,0)
all_images =  np.vstack([np.array(all_alphabets[key]) for key in all_alphabets])

In [9]:
def flip(p):
    # bigger p = less TRUE
    # True - in same class
    return random.random() < p
def generator(features,batch_size,distribution):

    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 2,105, 105,1))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            if flip(distribution):
                indecies= np.random.choice(len(all_images),2)
            else:
                first = np.searchsorted(lengthes, np.random.randint(1,len(all_images)))
                indecies = np.random.choice(range(lengthes[first-1],lengthes[first]),2)
            #print(indecies)
            image1,image2 = all_images[indecies[0]].reshape(105,105,1),all_images[indecies[1]].reshape(105,105,1)
            batch_features[i] = (image1,image2)
            distance1,distance2 = lengthes-indecies[0],lengthes-indecies[1]
            not_insame_class = np.argmax(distance1>0) != np.argmax(distance2>0)
            batch_labels[i] = not_insame_class
        yield [batch_features[:,0],batch_features[:,1]], batch_labels

In [ ]:
final_model.fit_generator(generator(all_images,32,0.75),
        steps_per_epoch=15,
        verbose=2,
        epochs=50)

Epoch 1/50
